In [ ]:
pip install pyswarms

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pyswarms as ps
import time as time
from sklearn.svm import SVC


In [ ]:
# set waktu mulai
start=time.time()

In [ ]:
#memanggil data set
datafix = pd.read_csv('/content/drive/MyDrive/farhann/insurance.csv')

In [ ]:
# mencari data apakah ada yang kosong atau tidak
datafix.isnull().sum()

In [ ]:
datafix.info()

In [ ]:
dataselec = datafix.drop(['FraudFound', 'Year', 'PolicyNumber', 'PolicyType'], 1)

In [ ]:
dataselec.info()

In [ ]:
#merubah data class yang tadinya positif negatif ke 1 dan 2
X = np.array(datafix.drop(['FraudFound', 'Year', 'PolicyNumber', 'PolicyType'], 1))
y = np.array(datafix['FraudFound'])

In [ ]:
X

In [ ]:
y

In [ ]:
print(X.shape)

In [ ]:
clf = SVC(kernel='linear')

In [ ]:
    Xku = np.array(datafix.drop(['FraudFound', 'Year', 'PolicyNumber', 'PolicyType'], axis=1))
    year = np.array(datafix['Year'])
    yku = np.array(datafix['FraudFound'])


    # Proses PSO
    # Define objective function
    def f_per_particle(m, alpha):
        total_features = n_feature
        # Get the subset of the features from the binary mask
        if np.count_nonzero(m) == 0:
            X_subset = Xku
        else:
            X_subset = Xku[:, m == 1]

        # Perform classification and store performance in P
        X_trainsubset, X_testsubset, y_trainsubset, y_testsubset = train_test_split(X_subset, yku, test_size=0.2, random_state=5)
        clf.fit(X_trainsubset, y_trainsubset)
        P = (clf.predict(X_testsubset) == y_testsubset).mean()
        # Compute for the objective function
        j = (alpha * (1.0 - P) + (1.0 - alpha) * (1 - (X_subset.shape[1] / total_features)))

        return j


    def f(Xku, alpha=0.88):
        n_particles = Xku.shape[0]
        j = [f_per_particle(Xku[i], alpha) for i in range(n_particles)]

        return np.array(j)

    # inisialisai parameter
    # options = {'c1': 0.5, 'c2': 0.5, 'w': 0.72, 'k': 30, 'p': 3}
    options = {'c1': 2, 'c2': 2, 'w': 0.9, 'k': 10, 'p': 2}

    # Mengambil instance hasil PSO
    n_sample, n_feature = Xku.shape
    dimensions = n_feature  # dimensions should be the number of features
    optimizer = ps.discrete.BinaryPSO(n_particles=50, dimensions=dimensions, options=options)

    # Proses Optimisasi
    cost, pos = optimizer.optimize(f, iters=100)

In [ ]:
    attr = ["Month","WeekOfMonth","DayOfWeek","Make","AccidentArea","DayOfWeekClaimed","MonthClaimed","WeekOfMonthClaimed","Sex","MaritalStatus","Age","Fault","VehicleCategory","VehiclePrice","RepNumber","Deductible","DriverRating","Days:Policy-Accident","Days:Policy-Claim","PastNumberOfClaims","AgeOfVehicle","AgeOfPolicyHolder","PoliceReportFiled","WitnessPresent","AgentType","NumberOfSuppliments","AddressChange-Claim","NumberOfCars","BasePolicy"]
    attrArr = np.array(attr)
    tempAttrSelected = attrArr[pos == 1]
    attrSelected = pd.DataFrame(tempAttrSelected)
    nAttr = attrSelected.shape[0]
    tempDataPSO = Xku[:, pos == 1]
    dataPSO = pd.DataFrame(tempDataPSO)  # pembuatan data frame dari fitur terpilih
    instancePSO = len(dataPSO)  # jumlah instance fitur terpilih

    # Simpan data hasil PSO
    dataPSO.columns = attrArr[pos == 1]
    dataPSO['Year'] = pd.Series(year)
    dataPSO['FraudFound'] = pd.Series(yku)
    dataPSO.to_csv('/content/drive/MyDrive/farhann/HasilPSO.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
    X_train_PSO = dataPSO.loc[dataPSO['Year'].isin(['1994', '1995'])].drop(['FraudFound', 'Year'], axis=1)
    X_test_PSO = dataPSO[dataPSO['Year'] == 1996].drop(['FraudFound', 'Year'], axis=1)
    y_train_PSO = dataPSO['FraudFound'].loc[dataPSO['Year'].isin(['1994', '1995'])]
    y_test_PSO = dataPSO['FraudFound'].loc[dataPSO['Year'] == 1996]

    # X_train_PSO, X_test_PSO, y_train_PSO, y_test_PSO = train_test_split(X, y, test_size=0.2, random_state=5)

    # Prediksi hasil PSO
    clf.fit(X_train_PSO, y_train_PSO)
    y_pred_PSO = clf.predict(X_test_PSO)
    cm_PSO = confusion_matrix(y_test_PSO, y_pred_PSO)



    # Hasil PSO
    TP2 = (cm_PSO[0, 0])
    TN2 = (cm_PSO[1, 1])
    FP2 = (cm_PSO[1, 0])
    FN2 = (cm_PSO[0, 1])
    totalPSO = sum(sum(cm_PSO))
    sensitivityPSO = (cm_PSO[0, 0]/(cm_PSO[0, 0]+cm_PSO[0, 1]))*100
    specificityPSO = (cm_PSO[1, 1]/(cm_PSO[1, 1]+cm_PSO[1, 0]))*100
    precisionPSO = (cm_PSO[0, 0]/(cm_PSO[0, 0]+cm_PSO[1, 0]))*100
    NPVPSO = (cm_PSO[1, 1]/(cm_PSO[0, 1]+cm_PSO[1, 1]))*100
    accuracyPSO = ((cm_PSO[0, 0]+cm_PSO[1, 1])/total)*100
